In [7]:
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from copulae1 import *
from KDEs import *
from toolbox import *
import warnings
import itertools
import os
warnings.filterwarnings("ignore")

data = pd.read_csv('../processed_data/btc_future_crix.csv')
data.head()

ecdf_brr = ECDF(data.return_brr)
ecdf_btc = ECDF(data.return_btc)

u = ecdf_brr(data.return_brr)
v = ecdf_btc(data.return_btc)

brr = np.array(data.return_brr)
btc = np.array(data.return_btc)

kde_brr = KDE(data.return_brr, "Gaussian")
kde_btc = KDE(data.return_btc, "Gaussian")

kde_brr = KDE(data.return_brr, "Gaussian", kde_brr.h_brot*3)
kde_btc = KDE(data.return_brr, "Gaussian", kde_btc.h_brot*3)

In [8]:
# Gaussian, t_Copula, Clayton, Frank, Gumbel, Plackett, Gaussian mix Indep
C1  = Gaussian(dict(rho=0.9),       Law_RS=kde_brr, Law_RF=kde_btc) # fix the maringals!
C2  = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=2) 
C2c = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=4) 
C3  = Clayton(dict(theta=0.1),      Law_RS=kde_brr, Law_RF=kde_btc)
C4  = Frank(dict(theta=0.1),        Law_RS=kde_brr, Law_RF=kde_btc)
C5  = Gumbel(dict(theta=3),         Law_RS=kde_brr, Law_RF=kde_btc)
C6  = Plackett(dict(theta=10),      Law_RS=kde_brr, Law_RF=kde_btc)
C7  = Gaussian_Mix_Independent(dict(rho=.5,p=0.7),Law_RS=kde_brr, Law_RF=kde_btc)
Copulae_names = ['Gaussian', 't_Copula', 't_Copula_Capped', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']
Copulae_arr   = [C1,C2,C2c,C3,C4,C5,C6,C7]
Copulae = dict(zip(Copulae_names, Copulae_arr))


In [9]:
data_name = "future_brr_v3"
data_path = "../processed_data/"+data_name+"/"

# Risk Measures
# Variance is automatically included
k_arr = [10,20] # Absolute risk aversion for exponential risk measure
q_arr_ES = [0.01,0.05,0.1] # Quantile level for expected shortfall

spot_name = "log return brr"
future_name = "log return future"

paras_results = []
likelihood_results = []
best_h_results = []

calibration_method = "MM" # MM or MLE
q_arr = [0.05,0.1,0.9,0.95]

ls = os.listdir(data_path+'train/')
ls = [l for l in ls if l.endswith('.csv')]
for file in ls:
    # Calibration 
    train = pd.read_csv(data_path+'train/'+file)
    spot   = train.loc[:,spot_name]
    future = train.loc[:,future_name]
    u = ECDF(spot)(spot)
    v = ECDF(future)(future)
    
    kde_brr = KDE(spot, "Gaussian")
    kde_btc = KDE(future, "Gaussian")
    
    for C_name in Copulae:
        Copulae[C_name].Law_RS = kde_brr
        Copulae[C_name].Law_RF = kde_btc
    
    paras = []
    likelihood = []
    best_h = []
    for C_name in Copulae:
        if calibration_method == "MLE":
            Copulae[C_name].canonical_calibrate(u,v)
            
        elif calibration_method == "MM":
            Copulae[C_name].mm_calibrate(u,v,q_arr)

        print(C_name,'is done.\n')
    
    for C_name in Copulae:
        paras.append((C_name,Copulae[C_name].paras))
        
    for C_name in Copulae:
        ln = Copulae[C_name].dependency_likelihood(u,v)
        likelihood.append((C_name,ln))
        
    paras_results.append(paras)
    likelihood_results.append(likelihood)
    
    # Get Best h
    best_h = []
    for C_name in Copulae:
        best_h.append(optimize_h(Copulae[C_name], k_arr, q_arr_ES))
    best_h = pd.DataFrame(best_h)
    best_h.columns = ['Var'] + ['ERM k=%i'%k for k in k_arr] +  ['ES q=%.2f'%q for q in q_arr_ES]
    best_h.index = Copulae_names 
    best_h_results.append(best_h)
    
#     # Testing
#     test = pd.read_csv(data_path+'test/'+file)
#     spot_test   = train.loc[:,spot_name]
#     future_test = train.loc[:,future_name]

Optimization terminated successfully.
         Current function value: 0.020007
         Iterations: 33
         Function evaluations: 68
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.011028
         Iterations: 128
         Function evaluations: 264
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.012225
         Iterations: 170
         Function evaluations: 328
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.350726
         Iterations: 44
         Function evaluations: 96
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.100847
         Iterations: 48
         Function evaluations: 110
Frank is done.

Optimization terminated successfully.
         Current function value: 0.080658
         Iterations: 35
         Function evaluations: 76
Gumbel is done.

Optimization terminated successfully.
         Curre

Optimization terminated successfully.
         Current function value: 0.016778
         Iterations: 133
         Function evaluations: 252
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.213357
         Iterations: 34
         Function evaluations: 74
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.097436
         Iterations: 38
         Function evaluations: 84
Frank is done.

Optimization terminated successfully.
         Current function value: 0.151122
         Iterations: 33
         Function evaluations: 72
Gumbel is done.

Optimization terminated successfully.
         Current function value: 0.045624
         Iterations: 40
         Function evaluations: 89
Plackett is done.

Optimization terminated successfully.
         Current function value: 0.018396
         Iterations: 67
         Function evaluations: 138
Gauss Mix Indep is done.

Optimization terminated successfully.
         

Optimization terminated successfully.
         Current function value: 0.010977
         Iterations: 65
         Function evaluations: 140
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.000610
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.037762
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.052173
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 0.009951
         Iterations: 15
         Function evaluations: 34
Optimization terminated successfully.
         Current function value: 0.000644
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: -0.000566
         Iterations: 15
         Function evaluations: 31
Optimi

Optimization terminated successfully.
         Current function value: 0.039235
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.054346
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 0.010173
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: 0.001361
         Iterations: 16
         Function evaluations: 32
Optimization terminated successfully.
         Current function value: 0.000200
         Iterations: 17
         Function evaluations: 36
Optimization terminated successfully.
         Current function value: 0.000650
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.040668
         Iterations: 14
         Function evaluations: 28
Optimization terminated successful

Optimization terminated successfully.
         Current function value: 0.000092
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: -0.002295
         Iterations: 15
         Function evaluations: 31
Optimization terminated successfully.
         Current function value: 0.000598
         Iterations: 12
         Function evaluations: 24
Optimization terminated successfully.
         Current function value: 0.036834
         Iterations: 12
         Function evaluations: 24
Optimization terminated successfully.
         Current function value: 0.052551
         Iterations: 12
         Function evaluations: 24
Optimization terminated successfully.
         Current function value: 0.008109
         Iterations: 13
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.000466
         Iterations: 14
         Function evaluations: 30
Optimization terminated successfu

...

Optimization terminated successfully.
         Current function value: 0.004195
         Iterations: 15
         Function evaluations: 32
Optimization terminated successfully.
         Current function value: 0.001020
         Iterations: 16
         Function evaluations: 33
Optimization terminated successfully.
         Current function value: 0.074080
         Iterations: 31
         Function evaluations: 66
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.057304
         Iterations: 122
         Function evaluations: 225
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.062315
         Iterations: 131
         Function evaluations: 247
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.577094
         Iterations: 36
         Function evaluations: 82
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.169479
      

Optimization terminated successfully.
         Current function value: 0.055217
         Iterations: 133
         Function evaluations: 259
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.063121
         Iterations: 130
         Function evaluations: 248
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.570970
         Iterations: 37
         Function evaluations: 82
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.176621
         Iterations: 37
         Function evaluations: 84
Frank is done.

Optimization terminated successfully.
         Current function value: 0.052787
         Iterations: 33
         Function evaluations: 70
Gumbel is done.

Optimization terminated successfully.
         Current function value: 0.106663
         Iterations: 39
         Function evaluations: 89
Plackett is done.

Optimization terminated successfully.
         Curren

Optimization terminated successfully.
         Current function value: 0.060523
         Iterations: 62
         Function evaluations: 133
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.000584
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.033442
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.043737
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.009606
         Iterations: 14
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 0.000974
         Iterations: 14
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: -0.001579
         Iterations: 15
         Function evaluations: 32
Optimi

Optimization terminated successfully.
         Current function value: 0.057536
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 0.013804
         Iterations: 15
         Function evaluations: 32
Optimization terminated successfully.
         Current function value: 0.002534
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: 0.000103
         Iterations: 16
         Function evaluations: 33
Optimization terminated successfully.
         Current function value: 0.001038
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.048281
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.065507
         Iterations: 14
         Function evaluations: 28
Optimization terminated successful

Optimization terminated successfully.
         Current function value: -0.000361
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: 0.000948
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.046226
         Iterations: 13
         Function evaluations: 26
Optimization terminated successfully.
         Current function value: 0.063815
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 0.013058
         Iterations: 14
         Function evaluations: 29
Optimization terminated successfully.
         Current function value: 0.002351
         Iterations: 14
         Function evaluations: 31
Optimization terminated successfully.
         Current function value: 0.000106
         Iterations: 16
         Function evaluations: 35
Optimization terminated successfu

In [10]:
s = Copulae[C_name].sample(10000000)

In [11]:
size = 100000000
samples = np.ones((size, 2))
u1 = stats.uniform.rvs(size=size)
v = stats.uniform.rvs(size=size)
u2 = Copulae[C_name].D1C_inv(u1, v)


AttributeError: 'Gaussian_Mix_Independent' object has no attribute 'D1C_inv'

In [ ]:
u2[u2<0] 

In [ ]:
c_arr = []
date_range_arr = []
for i, file in enumerate(ls):
    train = pd.read_csv(data_path+'train/'+file)
    date_range = train.Date.iloc[-1] +' to ' + train.Date.iloc[0]
    date_range_arr.append(date_range)
    
    c = pd.DataFrame(paras_results[i])
    c.index = c.iloc[:,0]
    c = pd.DataFrame(c.iloc[:,1])
#     c.columns = ['paras']
    c_arr.append(c)
    
paras_results_pd = pd.concat(dict(zip(ls, c_arr)), axis=1)
paras_results_pd.columns = paras_results_pd.columns.droplevel(1)
paras_results_pd.index.name= None

l_arr = []
date_range_arr = []
for i, file in enumerate(ls):
    train = pd.read_csv(data_path+'train/'+file)
    date_range = train.Date.iloc[-1] +' to ' + train.Date.iloc[0]
    date_range_arr.append(date_range)
    
    c = pd.DataFrame(likelihood_results[i])
    c.index = c.iloc[:,0]
    c = pd.DataFrame(c.iloc[:,1])
#     c.columns = ['likelihood']
    l_arr.append(c)
    
likelihood_results_pd = pd.concat(dict(zip(ls, l_arr)), axis=1)
likelihood_results_pd.columns = likelihood_results_pd.columns.droplevel(1)
likelihood_results_pd.index.name= None

In [ ]:
paras_results_pd

In [ ]:
likelihood_results_pd

In [ ]:
best_h_results_pd = pd.concat(dict(zip(ls, best_h_results)), axis=1)

In [ ]:
def cap(h):
    if h < 0:
        return 0
    elif h >1:
        return 1
    else:
        return h

In [ ]:
columns = best_h_results_pd.columns
for c in columns:
     best_h_results_pd.loc[:,c] = best_h_results_pd.loc[:,c].apply(cap)

In [ ]:
def hedging_effectiveness(h_arr, spot, future, k_arr, q_arr):
    results = np.ones((len(h_arr),1+len(k_arr)+len(q_arr)))
    for i, h in enumerate(h_arr):
        rh = spot-h*future
        results[i,:]=1-risk_measures(k_arr, q_arr, rh)/risk_measures(k_arr, q_arr, spot)
    return np.array([results[i,i] for i in range(len(h_arr))])

In [ ]:
date_range_arr = []
for i, file in enumerate(ls):
    train = pd.read_csv(data_path+'train/'+file)
    date_range = train.Date.iloc[-1] +' to ' + train.Date.iloc[0]
    date_range_arr.append(date_range)

In [ ]:
display_likelihood = likelihood_results_pd.copy()
display_likelihood.columns = date_range_arr
display_likelihood = display_likelihood.reindex(sorted(display_likelihood.columns), axis=1)

In [ ]:
display_paras = paras_results_pd.copy()
display_paras.columns = date_range_arr
display_paras = display_paras.reindex(sorted(display_paras.columns), axis=1)

In [ ]:
display_best_h = best_h_results_pd.copy()
display_best_h.columns.set_levels(date_range_arr, level=0, inplace=True)
display_best_h = display_best_h.reindex(sorted(display_best_h.columns), axis=1)

In [ ]:
if os.path.exists("../results/" + data_name)==False:
    print("Create new folder for results")
    os.mkdir("../results/" + data_name)
    os.mkdir("../results/" + data_name +"/MLE")
    os.mkdir("../results/" + data_name +"/MM")
    
if calibration_method == "MLE":
    path = "../results/"+ data_name +"/MLE/"
    paras_results_pd.to_json(path+"parameters.json")
    likelihood_results_pd.to_json(path+"likelihood.json")
    best_h_results_pd.to_json(path+"best_h.json")
    
    display_paras.to_html(path+"paras.html")
    display_likelihood.to_html(path+"likelihood.html")
    display_best_h.to_html(path+"best_h.html")

    
elif calibration_method == "MM":
    path = "../results/"+ data_name +"/MM/"
    paras_results_pd.to_json(path+"parameters.json")
    likelihood_results_pd.to_json(path+"likelihood.json")
    best_h_results_pd.to_json(path+"best_h.json")
    
    display_paras.to_html(path+"paras.html")
    display_likelihood.to_html(path+"likelihood.html")
    display_best_h.to_html(path+"best_h.html")

In [ ]:
HE_results = []

for i, file in enumerate(ls):
    test = pd.read_csv(data_path+'train/'+file)
    spot   = test.loc[:,spot_name]
    future = test.loc[:,future_name]
    fn = lambda h_arr: hedging_effectiveness(h_arr, spot, future, k_arr, q_arr)
    HE = pd.DataFrame().reindex_like(best_h_results[0])
    HEs = best_h_results[i].apply(fn, axis=1)
    for i in range(len(HEs)):
        HE.iloc[i,:] = HEs[i]
    HE_results.append(HE)
HE_results_pd = pd.concat(dict(zip(ls, HE_results)), axis=1)
risk_measure_names = HE_results_pd.droplevel(0,axis=1).columns.unique()

fig, ax = plt.subplots(len(risk_measure_names), 1, figsize=(10,5*len(risk_measure_names)))
for i, name in enumerate(risk_measure_names): 
    ax[i].boxplot(HE_results_pd.droplevel(0,axis=1).loc[:,name])
    ax[i].set_xticklabels(Copulae_names)
    ax[i].set_title("In Sample Hedging Effectiveness of %s"%name)
    
if calibration_method == "MLE":
    fig.savefig("../results/"+ data_name +"/MLE/"+"In Sample Hedging Effectiveness.png", transparent=True)    
elif calibration_method == "MM":
    fig.savefig("../results/"+ data_name +"/MM/"+"In Sample Hedging Effectiveness.png", transparent=True)

In [ ]:
HE_results = []

for i, file in enumerate(ls):
    test = pd.read_csv(data_path+'test/'+file)
    spot   = test.loc[:,spot_name]
    future = test.loc[:,future_name]
    fn = lambda h_arr: hedging_effectiveness(h_arr, spot, future, k_arr, q_arr)
    HE = pd.DataFrame().reindex_like(best_h_results[0])
    HEs = best_h_results[i].apply(fn, axis=1)
    for i in range(len(HEs)):
        HE.iloc[i,:] = HEs[i]
    HE_results.append(HE)
HE_results_pd = pd.concat(dict(zip(ls, HE_results)), axis=1)
risk_measure_names = HE_results_pd.droplevel(0,axis=1).columns.unique()

fig, ax = plt.subplots(len(risk_measure_names), 1, figsize=(10,5*len(risk_measure_names)))
for i, name in enumerate(risk_measure_names): 
    ax[i].boxplot(HE_results_pd.droplevel(0,axis=1).loc[:,name])
#     ax[i].set_xticks(np.linspace(1,len(Copulae_names)))
    ax[i].set_xticklabels(Copulae_names)
    ax[i].set_title("Out of Sample Hedging Effectiveness of %s"%name)
if calibration_method == "MLE":
    fig.savefig("../results/"+ data_name +"/MLE/"+"Out of Sample Hedging Effectiveness.png", transparent=True)    
elif calibration_method == "MM":
    fig.savefig("../results/"+ data_name +"/MM/"+"Out of Sample Hedging Effectiveness.png", transparent=True)